# Расшифровка протокола беспроводного устройства

В наше время существует великое множество различных беспроводных девайсов: выключатели, датчики температуры и влажности, просто лампочки и т. д.
 
Как правило такие устройства поставляются с какими-либо приемниками: пультами для выключателей, метеостанциями принимающими сигналы датчиков или универсальными приемо-передачиками.

Однако не всегда хочется работать с устройствами стандартным, одобренным производителем, способом. Особенно это проявляется, если есть несколько устройств от разных производителей с различными протоколами, а управлять всем этим добром хочется централизованно.

В этом случае появляется желание разобрать протокол девайса и научить центральное устройство его понимать.

Итак больше конкретики: в моем случае есть WirenBoard 5 и китайский датчик движения PA-92R на чипе HS2241.



![motion sensor](./motion_sensor.jpg "Motion sensor")

Нужно разобрать протокол. Первое что для этого стоит сделать - получить какие-нибудь реальные данные о протоколе, 
более конкретно сделать дамп эфира.

Для этого воспользуемся драйвером радио `wb-homa-rfsniffer` из репозитория WirenBoard. 

Остановим его как сервис и запустим в режиме записи эфира.

`service wb-homa-rfsniffer stop`

`{some path}/wb-homa-rfsniffer -W`

После начала записи пытаемся вызвать всевозможные сигналы от устройства. В нашем случае просто несколько раз машем рукой перед датчиком. Потом разбираем корпус переставляем джамперы задающие адрес устройства и еще раз машем.

Останавливаем запись эфира. Теперь мы имеем `.rcf`-файл с различными образцами протокола. Осталось его проанализировать и научиться парсить.

Встает вопрос: какие инструменты для этого использовать? В качеcтве среды я выбрал Jupyter notebook, как удобный инструмент для интерактивных вычислений. Язык - python, чтобы кратко писать сложные вещи. К тому же в нем есть удобные инструменты анализа и визуализации данных: scipy, pandas, matplotlib.

Весь код анализа данных приведен [здесь](ссылка). Советую читать его параллельно с дальнейшим текстом.

## Анализируем данные

Посмотрим на первые байты эфира.

`128.95.0.1.198.3.0.0.71.48.55.1.188.1.0.0.126.1.0.1.191.4.0.0.111.1.0.1.18.5.0.0`

Видим периодически повторяющиеся пары нулей. А если так?

`128.95.0.1  198.3.0.0  71.48.55.1  188.1.0.0  126.1.0.1  191.4.0.0  111.1.0.1  18.5.0.0`

Дальше уже несложно догадаться, что импульсы и паузы кодируются 4 байтами каждый, при этом последний байт говорит нам импульс это или пауза, а первые три байта задают длину. Эмпирически можно выяснить что в микросекундах.
Представим сигналы в виде чисел, где знак различает импульс и паузу, а модуль равен продолжительности сигнала.

`    +24448       -966    +3616839       -444       +382      -1215       +367     -1298`

Так уже невооруженным глазом можно что-то понимать. Однако даже понять где начинается и кончается сообщение датчика все еще очень сложно. Попробуем поискать смысл в этих сигналах. Начнем с самого простого метода анализа: построим гистограмму.

![pulses](./pulses.png "Pulses")
![pauses](./pauses.png "Pauses")

Главное, что отсюда можно вынести - есть 5 групп сигналов. Ведь вряд ли два сигнала из одного пика имеют разный смысл. 

Сделаем над нашими данными такое преобразование: заменим числовой код сигналов на латинские буквы соответствующие пикам на гистограммах. При этом импульсы заменим на заглавные буквы, а паузы на строчные. Порядок букв сделаем по возрастанию длин сигналов. Сигналы, не вошедшие ни в какую группу, будут '?'.

Определять границы групп сигналов можно вручную. А можно автоматически кластеризовать, используя scipy (см. ipython ноутбук).

В результате получим что-то такое:

`bBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbBaAbBaAbAcAbAbBaAbAbBaAbAbAb` <br>
`BaBaAbBaBaAbAbBaAbBaAbBaAbBaAbAcAbAbBaAbAbBaAbAbAbBaBaAbBaB` <br>
`aAbAbBaAbBaAbBaAbBaAbAcAbAbBaAbAbBaAbAbAbBaBaAbBaBaAbAbBaAb` <br>
`BaAbBaAbBaAbAcAb............`

Ничего непонятно. Видимо нужно немного изменить способ записи. Снова посмотрим на гистограмму и обратим внимание на маленький пик длинных пауз. Вполне возможно, что этот пик содержит паузы, служащие разделителями в сообщении датчика. Запишем еще раз, но переводы строк сделаем перед символами 'c'.

`bBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbBaAbBaAbA` <br>
`cAbAbBaAbAbBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbBaAbBaAbA` <br>
`cAbAbBaAbAbBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbBaAbBaAbA` <br>
`cAbAbBaAbAbBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbBaAbBaAbA` <br>
`cAb............`

Вот такой поворот. Тут повторяется одно и тоже. Становится понятным способ проверки целостности данных. Просто последовательный прием пакетов и их сравнение. Если совпадают, то сообщение корректно.

Теперь возьмем все данные, порубим по 'c' как по разделителю и выведем все уникальные строки. Получим:

[редко повторяющиеся строки разных длин пропустим]  <br>
'AbAbBaAbAbBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbAbBaBaAbA' - 98 повторений  <br>
'AbAbBaAbAbBaAbAbAbBaBaAbBaBaAbAbBaAbBaAbBaAbBaAbA' - 74 повторения <br>

Когда я записывал дамп эфира, то один раз переставлял джамперы. Вернее один джампер с первой позиции на нулевую.
Разницы в сообщениях у нас "AbBa" -> "BaAb". Откуда можно сделать предположение, что "Ab" кодирует 1, а "Ba" - 0.
Попробуем декодировать таким образом. Получим:

'001001000110110010100110' <br>
'001001000110110010101010'

Здесь последний символ 'A' отбрасываем. Или лучше так: считаем 'Ac' разделителем пакетов.
Кажется, уже почти все. Осталось понять, что значат эти нолики и единички. 
Про последние 4 все просто. 0-3 джампер на плате дословно задают последние 4 бита.
А первые 20 бит у меня ни разу не менялись. Так что можно только строить предположения...

Или найти даташит (да с этого надо было начинать, но так же неинтересно ;) ).

Впрочем, последнее ни разу не тривиально. Я лично так и не смог его найти. Но, к счастью, мне подсказали где посмотреть даташит на похожий чип (http://sc-tech.cn/en/hs1527.pdf). Оказалось, что все сделанные выводы верны. А первые 20 бит это просто вшитый адрес.


## Итоги

Был разобран довольно простой протокол, я постарался сделать это как можно более понятно. Надеюсь, что вы вынесете отсюда что-то новое для себя и, если придется, не побоитесь исследовать непонятный набор ноликов и единичек.
